In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd

import kraft

In [ ]:
element_x_node = pd.read_csv("w.tsv", sep="\t", index_col=0)

element_x_node.columns.name = "Factor"

In [ ]:
gps_map = kraft.GPSMap(element_x_node)

In [ ]:
gps_map.plot()

In [ ]:
gps_map.plot(elements_to_highlight=("Feature 0", "Feature 1"))

In [ ]:
element_value = gps_map.element_x_dimension.sum(axis=1)

element_value.name = "Element Value"

gps_map.plot(element_value=element_value)

element_value[::2] = np.nan

gps_map.plot(element_value=element_value)

In [ ]:
gps_map.set_element_label(
    pd.Series(
        gps_map.element_x_dimension.values.argmax(axis=1),
        index=element_x_node.index,
        name="Element Label",
    )
)

gps_map.plot()

In [ ]:
gps_map.plot(element_value=element_value)

In [ ]:
gps_map.predict(
    gps_map.element_x_node.iloc[:8, :], element_trace={"marker": {"size": 32}}
)

In [ ]:
h = pd.read_csv("kras_gps_map.h.tsv", sep="\t", index_col=0)

h.columns.name = "Cell Line"

h = kraft.normalize_dataframe(
    kraft.normalize_dataframe(h, 1, "-0-").clip(lower=-3, upper=3), 1, "0-1"
)

gps_map = kraft.GPSMap(h.T, pull_power=2)

gps_map.set_element_label(
    pd.read_csv("kras_gps_map.k_x_h_element.tsv", sep="\t", index_col=0).loc["K15"]
)

gps_map.plot()

In [ ]:
h_ = h.copy()

h_.columns.name = "New {}".format(h.columns.name)

gps_map.predict(h_.T, pull_power=2)

In [ ]:
gps_map_path = "output/gps_map.pickle.gz"

kraft.write_gps_map(gps_map, gps_map_path)

In [ ]:
gps_map = kraft.read_gps_map(gps_map_path)

gps_map.plot()